# Gráficos de Área, Barras e Histogramas

## Objetivos

Después de completar este laboratorio será capas de:

*   Crear gráficos de área, de barras e  histogramas.


## Tabla de Contenidos

1.  [Explorando Conjunto de Datos con *pandas* y *Matplotlib*](#0)
2.  [Descarga y Preparación de Datos](#2)
3.  [Visualización de datos usando Matplotlib](#4)
4.  [Gráfico de Áreas](#6)
5.  [Histogramas](#8)
6.  [Gráfico de Barras](#10)

# Explorando Conjunto de Datos con *pandas* y *Matplotlib*<a id="0"></a>

Kit de herramientas: El curso se basa en gran medida en [**pandas**](http://pandas.pydata.org) y [**Numpy**](http://www.numpy.org/) para la manipulación, el análisis y la visualización de datos. La biblioteca principal de visualización que se está explorando en el curso es [**Matplotlib**](http://matplotlib.org/).

Conjunto de datos: Inmigración a Canada desde 1980 a 2013 - [International migration flows to and from selected countries - The 2015 revision](http://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp) desde el sitio ewb de las Naciones Unidas.

El conjunto de datos contiene datos anuales sobre los flujos de migrantes internacionales registrados por los países de destino. Los datos presentan tanto las entradas como las salidas según el lugar de nacimiento, ciudadanía o lugar de residencia anterior/próxima tanto para extranjeros como para nacionales. Este laboratorio se concentra en los datos de inmigración canadiense.


# Descarga y Preparación de Datos <a id="2"></a>


Lo primero que hay que realizar es la instalación de **openpyxl** (antiguamente conocido como **xlrd**) que es un módulo que **pandas** requiere para leer archivos Excel.


In [ ]:
!pip install openpyxl

A continuación, se procederá a cargar los dos módulos claves para hacer análisis de datos **pandas** y
 **numpy**.

In [ ]:
import numpy as np  # util para computación científica en Python
import pandas as pd # biblioteca que contienen la estructura de datos de uso principal

Ahora, se carga el conjunto de datos con los datos de la inmigración en Canadá usando el método de **pandas** `read_excel()`.


In [ ]:
df_can = pd.read_excel(
    'datos/Canada.xlsx',
    sheet_name='Canada by Citizenship',
    skiprows=range(20),
    skipfooter=2)

print('Datos leídos y cargados en un dataframe de pandas!')

Mostrar las primeras 5 filas del conjunto de datos usando la función `head()`.

In [ ]:
df_can.head()

Descubrir cuantas entradas hay en el conjunto de datos.


In [ ]:
# dimensión del dataframe (rows, columns)
df_can.shape    

Limpiar datos. Se realizarán algunas modificaciones al conjunto de datos original para que sea más fácil crear visualizaciones. La explicación de estas modificaciones están detalladas en el laboratorio anterior *"Introducción a Matplotlib y Gráficos de línea"*.

#### 1. Limpiar el conjunto de datos para eliminar las columnas que no son informativas para la visualización (por ejemplo, Tipo, AREA, REG).

In [ ]:
df_can.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)

# mostrar los primeros cinco elementos para ver cómo se cambió el dataframe
df_can.head()

Observar cómo las columnas Tipo, Cobertura, AREA, REG y DEV se eliminaron del dataframe.

#### 2. Cambiar el nombre de algunas de las columnas para que tengan sentido.

In [ ]:
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# mostrar los primeros cinco elementos para ver cómo se cambió el dataframe
df_can.head()

Observar cómo los nombres de las columnas ahora tienen mucho más sentido.

#### 3. Para mantener la coherencia, asegúrese de que todas las etiquetas de las columnas sean de tipo cadena de caracteres o string.

In [ ]:
# examinar los tipos de etiquetas de las columnas

all(isinstance(column, str) for column in df_can.columns)

Observar cómo la línea de código anterior devolvió *Falso* cuando se probó si todas las etiquetas de las columnas son del tipo **cadena**. Así que hay que cambiar todos al tipo **string**.


In [ ]:
df_can.columns = list(map(str, df_can.columns))

# comprobar el tipo de las etiquetas de las columnas
all(isinstance(column, str) for column in df_can.columns)

#### Establecer el nombre del país como índice. Esto es útil para buscar rápidamente países utilizando el método .loc.


In [ ]:
df_can.set_index('Country', inplace=True)

# mostrar los primeros cinco elementos para ver cómo se cambió el dataframe
df_can.head()

Observe ahora que los nombres de los países ahora sirven como índices.


#### 5. Agregar la columna total.


In [ ]:
df_can['Total'] = df_can.sum(axis=1, numeric_only=True)

# mostrar los primeros cinco elementos para ver cómo se cambió el dataframe
df_can.head()

Ahora el dataframe tiene una columna adicional que presenta el número total de inmigrantes de cada país en el conjunto de datos de 1980 a 2013. Así, al imprimir la dimensión de los datos, se obtiene.

In [ ]:
print('Dimensión de los datos:', df_can.shape)

Así, ahora el dataframe tiene 38 columnas en lugar de las 37 columnas que se tenía antes

In [ ]:
# finalmente, se crea una lista de años desde 1980 - 2013
# esto será útil cuando comencemos a graficar los datos
years = list(map(str, range(1980, 2014)))

years

# Visualización de datos usando Matplotlib<a id="4"></a>


Importar la biblioteca `matplotlib`.


In [ ]:
# se hará uso del despliegue de figuras en línea o dentro de Jupyter Notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot')  # opcional: para estilo similar a ggplot

# verificar la versión de Matplotlib
print('Matplotlib version: ', mpl.__version__)

# Gráfico de Áreas<a id="6"></a>


Anteriormente se creó un gráfico de líneas que mostró a los 5 países que más aṕrtaban con inmigrantes a Canadá desde 1980 a 2013. Con algunas pequeñas modificaciones en el código se puede mostrar una gráfica acumulativa conocida como **Gráficp de áreas apilado** or **Gráfico de áreas**.


In [ ]:
df_can.sort_values(['Total'], ascending=False, axis=0, inplace=True)

# obtener las 5 mejores entradas
df_top5 = df_can.head()

# transponer el dataframe
df_top5 = df_top5[years].transpose()

df_top5.head()

Los gráficos de área se apilan de forma predeterminada. Y para producir un gráfico de áreas apiladas, cada columna debe tener todos los valores positivos o negativos (para cualquier `NaN`, es decir, no un número, los valores predeterminados serán 0). Para producir un gráfico no apilado, establezca el parámetro `stacked` en el valor `False`.

In [ ]:
# se cambia los valores de índice de df_top5 para escribir un número entero para graficar
df_top5.index = df_top5.index.map(int)

df_top5.plot(kind='area',
             stacked=False,
             figsize=(20, 10))  # pasa un tamaño en la tupla (x, y)

plt.title('Tendencia de inmigración de los países Top 5')
plt.ylabel('Número of Inmigrantes')
plt.xlabel('Años')

plt.show()

El gráfico no apilado tiene una transparencia predeterminada (valor alpha) de 0.5. Podemos modificar este valor pasando el parámetro `alpha`.

In [ ]:
df_top5.plot(kind='area', 
             alpha=0.25,  # 0 - 1, valor por defecto alpha = 0.5
             stacked=False,
             figsize=(20, 10))

plt.title('Tendencia de inmigración de los países Top 5')
plt.ylabel('Número of Inmigrantes')
plt.xlabel('Años')

plt.show()

### Dos tipos de gráficos

Hay dos estilos/opciones de trazado con `matplotlib`, trazado usando la capa de artista y trazado usando la capa de secuencias de comandos.

\*\*Opción 1: Capa de secuencia de comandos (método procedural) - usando matplotlib.pyplot como 'plt' \*\*

Ahora se puede usar `plt` es decir, `matplotlib.pyplot` y agregar más elementos llamando diferentes métodos de manera procedural; por ejemplo, `plt.title(...)` para agregar el título o `plt.xlabel(...)` para agregar la etiqueta del eje x.

```python
    # Opción 1: Este es que se ha usando hasta ahora.
    df_top5.plot(kind='area', alpha=0.35, figsize=(20, 10)) 
    plt.title('Tendencia de inmigración de los países Top 5')
    plt.ylabel('Número of Inmigrantes')
    plt.xlabel('Años')
```


\*\*Opción 2: Capa de artista (Método orientado a objeto) - usando un `Axes` en vez de Matplotlib (preferido) \*\*

Puede usar una instancia de `Axes` del gráfico actual y almacenarla en una variable (por ejemplo, `ax`). Puede agregar más elementos llamando a métodos con un pequeño cambio en la sintaxis (agregando "`set_`" a los métodos anteriores). Por ejemplo, use `ax.set_title()` en lugar de `plt.title()` para agregar un título, o `ax.set_xlabel()` en lugar de `plt.xlabel()` para agregar una etiqueta al eje x.

Esta opción a veces es más transparente y flexible para usar en gráficos avanzados (en particular cuando se tienen múltiples gráficos, como se verá más adelante).

Este curso utilizará la **capa de secuencias de comandos**, a excepción de algunas visualizaciones avanzadas en las que necesitaremos usar la **capa de artista** para manipular aspectos avanzados de los gráficos.

In [ ]:
# opción 2: opción preferida con más flexibilidad
ax = df_top5.plot(kind='area', alpha=0.35, figsize=(20, 10))
print(type(ax))

ax.set_title('Tendencia de inmigración de los países Top 5')
ax.set_ylabel('Número of Inmigrantes')
ax.set_xlabel('Años')

**Pregunta**: utilice la capa de secuencias de comandos para crear un gráfico de áreas apiladas de los 5 países que menos contribuyeron a la inmigración a Canadá **desde** 1980 hasta 2013. Utilice un valor de transparencia de 0.45.

In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo





Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

#obtener las 5 países con menor contribución de inmigrantes
df_least5 = df_can.tail(5)
 
# transponer el dataframe
df_least5 = df_least5[years].transpose() 
df_least5.head()

df_least5.index = df_least5.index.map(int) # se cambia los valores de índice de df_top5 para escribir un número entero para graficar
df_least5.plot(kind='area', alpha=0.45, figsize=(20, 10)) 

plt.title('Tendencia de inmigración de los 5 países con menos inmigrantes')
plt.ylabel('Número of Inmigrantes')
plt.xlabel('Años')

plt.show()

-->

**Pregunta**: utilice la capa de artista para crear un diagrama de área no apilada de los 5 países que menos contribuyeron a la inmigración a Canadá **desde** 1980 hasta 2013. Use un valor de transparencia de 0.55.

In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo





Doble click **aquí** para ver la solución

<!--
#La respuesta correcta es:  
#obtener las 5 países con menor contribución de inmigrantes
df_least5 = df_can.tail(5)
 
# transponer el dataframe
df_least5 = df_least5[years].transpose() 
df_least5.head()

df_least5.index = df_least5.index.map(int) # se cambia los valores de índice de df_top5 para escribir un número entero para graficar

ax = df_least5.plot(kind='area', alpha=0.55, stacked=False, figsize=(20, 10))
ax.set_title('Tendencia de inmigración de los 5 países con menos inmigrantes')
ax.set_ylabel('Número of Inmigrantes')
ax.set_xlabel('Años')

-->    

# Histogramas<a id="8"></a>

Un histograma es una forma de representar la distribución de *frecuencia* de un conjunto de datos numéricos. La forma en que funciona es que divide el eje x en *contenedores*, asigna cada punto de datos en nuestro conjunto de datos a un contenedor y luego cuenta la cantidad de puntos de datos que se han asignado a cada contenedor. Entonces, el eje y es la frecuencia o la cantidad de puntos de datos en cada contenedor. Tenga en cuenta que podemos cambiar el tamaño del contenedor y, por lo general, es necesario modificarlo para que la distribución se muestre bien.

**Pregunta:** ¿Cuál es la distribución de frecuencias del número (población) de nuevos inmigrantes de varios países a Canadá en 2013?

Antes de continuar con la creación del gráfico de histograma, primero se examinará los datos divididos en intervalos. Para hacer esto, usaremos el método `histrogram` de **Numpy** para obtener los rangos de contenedores y los conteos de frecuencia de la siguiente manera:

In [ ]:
# Vista rápida de los datos de 2013
df_can['2013'].head()

In [ ]:
# np.histogram devuelve 2 valores
contador, bordes_contenedores = np.histogram(df_can['2013'])

print(contador) # Contador de frecuencias
print(bordes_contenedores) # rango de contenedores, por defecto = 10 contenedores

De forma predeterminada, el método `histrogram` divide el conjunto de datos en 10 contenedores. La siguiente tabla resume los rangos de intervalos y la distribución de frecuencia de la inmigración en 2013. Podemos ver que en 2013:

* 178 países aportaron entre 0 a 3412.9 inmigrantes
* 11 países aportaron entre 3412.9 a 6825.8 inmigrantes
* 1 país aportó entre 6285.8 a 10238.7 inmigrantes, y así sucesivamente.

|  | <sub><sup>**Cont. 1**</sup></sub> | <sub><sup>**Cont. 2**</sup></sub> | <sub><sup>**Cont. 3**</sup></sub> | <sub><sup>**Cont. 4**</sup></sub> | <sub><sup>**Cont. 5**</sup></sub> | <sub><sup>**Cont. 6**</sup></sub> | <sub><sup>**Cont. 7**</sup></sub> | <sub><sup>**Cont. 8**</sup></sub> | <sub><sup>**Cont. 9**</sup></sub> | <sub><sup>**Cont. 10**</sup></sub> |
|---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| <sub><sup>**Rango**</sup></sub> | <sub><sup>0 a 3412.9</sup></sub> | <sub><sup>3412.9 a 6825.8</sup></sub> | <sub><sup>68.25.8 a 10238.7</sup></sub> | <sub><sup>10238.7 a 13651.6</sup></sub>  | <sub><sup>13651.6 a 17064.5</sup></sub> | <sub><sup>17064.5 a 20477.4</sup></sub> | <sub><sup>20477.4 a 23890.3</sup></sub> | <sub><sup>23890.3 a 27303.2</sup></sub> | <sub><sup>27303.2 a 30716.1</sup></sub> | <sub><sup>30716.1 a 34129</sup></sub> |
| <sub><sup>**Frecu.**</sup></sub> | <sub><sup>178</sup></sub> | <sub><sup>11</sup></sub> | <sub><sup>1</sup></sub> | <sub><sup>12</sup></sub> | <sub><sup>0</sup></sub> | <sub><sup>0</sup></sub> | <sub><sup>0</sup></sub> | <sub><sup>0</sup></sub> | <sub><sup>1</sup></sub> | <sub><sup>2</sup></sub> |

Se puede graficar fácilmente esta distribución pasando `kind=hist` a `plot()`.


In [ ]:
df_can['2013'].plot(kind='hist', figsize=(8, 5))

# agregar un título al histograma
plt.title('Histograma de Inmigración de 195 Países en el Año 2013')
# agregar etiqueta al eje y
plt.ylabel('Número de Paises')
# agregar etiqueta al eje x
plt.xlabel('Número de Inmigrantes')

plt.show()

En el gráfico anterior, el eje x representa el rango de población de inmigrantes en intervalos de 3412,9. El eje y representa el número de países que contribuyeron a la población antes mencionada.

Observar que las etiquetas del eje x no coinciden con el tamaño del contenedor. Esto se puede solucionar pasando una palabra clave `xticks` que contiene la lista de tamaños de contenedores, de la siguiente manera:

In [ ]:
# 'bordes_contenedores' es una lista de intervalos de contenedores
contador, bordes_contenedores = np.histogram(df_can['2013'])

df_can['2013'].plot(kind='hist', figsize=(8, 5), xticks=bordes_contenedores)

plt.title('Histograma de Inmigración de 195 Países en el Año 2013') # agregar un título al histograma
plt.ylabel('Número de Paises') # agregar etiqueta al eje y
plt.xlabel('Número de Inmigrantes') # agregar etiqueta al eje x

plt.show()

*Nota:* Se podría usar `df_can['2013'].plot.hist()`, en su lugar. De hecho, a lo largo de este laboratorio, usar `algun_dato.plot(kind='tipo_grafico', ...)` es equivalente a `algun_dato.plot.tipo_grafico(...)`. Es decir, pasar el tipo de gráfico como argumento o método se comporta igual.

Ver la documentación de *pandas* para más información [http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html).


También se puede mostrar varios histogramas en el mismo gráfico. Por ejemplo, responder a las siguientes preguntas usando un histograma.

**Pregunta**: ¿Cuál es la distribución de inmigración para Dinamarca, Noruega y Suecia durante los años 1980 - 2013?

In [ ]:
# mostrar rápidamente el conjunto de datos
df_can.loc[['Denmark', 'Norway', 'Sweden'], years]

In [ ]:
# generar histogramas
df_can.loc[['Denmark', 'Norway', 'Sweden'], years].plot.hist()

¡El gráfico no tiene mucho sentido!

La solución radica en cómo se estructura el conjunto de datos a graficar.

En vez de graficar la distribución de frecuencia de la población de los 3 países, *pandas* en su lugar graficó la distribución de frecuencia de la población para los años `years`.

Esto se puede solucionar transponiendo primero el conjunto de datos y luego graficando como se muestra a continuación.


In [ ]:
# transponer el dataframe
df_t = df_can.loc[['Denmark', 'Norway', 'Sweden'], years].transpose()
df_t.head()

In [ ]:
# generar el histograma
df_t.plot(kind='hist', figsize=(10, 6))

plt.title('Histograma de Inmigración desde Dinamarca, Noruega, Suecia desde 1980 a 2013')
plt.ylabel('Número de Años')
plt.xlabel('Número de Inmigrantes')

plt.show()

Se realizarán algunas modificaciones para mejorar el impacto y la estética de la gráfica anterior:

*   Aumentar el tamaño del contenedor a agregando el parámetro`bins`;
*   Establecer la transparencia a un 60% agregando el parámetro`alpha`;
*   Etiquetar el eje x agregando el parámetro `x-label`;
*   Cambiar el color del gráfico agregando el parámetro `color`.


In [ ]:
# obtener los valores de marcas del eje x
contador, bordes_contenedores = np.histogram(df_t, 15)

# histograma no apilado
df_t.plot(kind ='hist', 
          figsize=(10, 6),
          bins=15,
          alpha=0.6,
          xticks=bordes_contenedores,
          color=['coral', 'darkslateblue', 'mediumseagreen']
         )

plt.title('Histograma de Inmigración desde Dinamarca, Noruega, Suecia desde 1980 a 2013')
plt.ylabel('Número de Años')
plt.xlabel('Número de Inmigrantes')

plt.show()

Consejo:
Para obtener una lista completa de los colores disponibles en Matplotlib, ejecute el siguiente código en su shell de python:

In [ ]:
import matplotlib
for nombre, hexadecimal in matplotlib.colors.cnames.items():
    print(nombre, hexadecimal)

Si no se quiere que las gráficas de los histogramas se superpongan entre sí, se pueden apilar usando el parámetro `stacked`. También se puede ajustar las etiquetas mínimo y máximo del eje x para eliminar el espacio adicional en los bordes del gráfico. Así se puede pasar una tupla (min, max) usando el parámetro `xlim`, como se muestra a continuación.

In [ ]:
contador, bordes_contenedores = np.histogram(df_t, 15)
xmin = bordes_contenedores[0] - 10   #  el valor del primer contenedor es 31.0, se descuenta un búfer de 10 por motivos estéticos 
xmax = bordes_contenedores[-1] + 10  #  el último valor del contenedor es 308.0, se agrega un búfer de 10 por motivos estéticos

# Histograma apilable
df_t.plot(kind='hist',
          figsize=(10, 6), 
          bins=15,
          xticks=bordes_contenedores,
          color=['coral', 'darkslateblue', 'mediumseagreen'],
          stacked=True,
          xlim=(xmin, xmax)
         )

plt.title('Histograma de Inmigración desde Dinamarca, Noruega, Suecia desde 1980 a 2013')
plt.ylabel('Número de Años')
plt.xlabel('Número de Inmigrantes')

plt.show()

**Pregunta**: Usar la capa de secuencias de comandos para mostrar la distribución de inmigración de Grecia, Albania y Bulgaria entre los años 1980 y 2013. Utilice un gráfico superpuesto o no apilable con 15 contenedores y un valor de transparencia de 0.35.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo





Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

# crear un dataframe de los países de interés (pdi)
df_pdi = df_can.loc[['Greece', 'Albania', 'Bulgaria'], years]

# transponer el dataframe
df_pdi = df_pdi.transpose() 

# obtener los valores de marcas del eje x
contador, bordes_contenedores = np.histogram(df_pdi, 15)

# Histograma no apilable
df_pdi.plot(kind ='hist',
            figsize=(10, 6),
            bins=15,
            alpha=0.35,
            xticks=bordes_contenedore,
            color=['red', 'blue', 'yellow']
            )

plt.title('Histograma de Inmigración desde Grecia, Albania y Bulgaria desde 1980 a 2013')
plt.ylabel('Número de Años')
plt.xlabel('Número de Inmigrantes')

plt.show()

-->

# Gráfico de Barras <a id="10"></a>

Un gráfico de barras es una forma de representar datos donde la *longitud* de las barras representa la magnitud/tamaño de la característica/variable. Los gráficos de barras generalmente representan variables numéricas y categóricas agrupadas en intervalos.

Para crear un gráfico de barras, podemos pasar uno de los dos argumentos disponibles a través del parámetro `kind` en `plot()`:


*   `kind=bar` crea un gráfico de barras *vertical*
*   `kind=barh` crea un gráfico de barras *horizontal*


**Gráfico de Barras Vertical**

En los gráficos de barras verticales, el eje x se usa para etiquetar, y la longitud de las barras en el eje y corresponde a la magnitud de la variable que se mide. Los gráficos de barras verticales son especialmente útiles para analizar datos de series temporales. Una desventaja es que carecen de espacio para el etiquetado de texto al pie de cada barra.

**Se empezará analizando el efecto de la crisis financiera de Islandia:**

La crisis financiera islandesa de 2008 - 2011 fue un evento económico y político importante en Islandia. En relación con el tamaño de su economía, el colapso bancario sistémico de Islandia fue el mayor experimentado por cualquier país en la historia económica. La crisis condujo a una grave depresión económica entre 2008 y 2011 ya un importante malestar político.

**Pregunta:** Comparar el número de inmigrantes islandeses (country = 'Iceland') en Canadá desde el año 1980 hasta el 2013.


In [ ]:
# paso 1: obtener los datos
df_islandia = df_can.loc['Iceland', years]
df_islandia.head()

In [ ]:
# paso 2: graficar los datos
df_islandia.plot(kind='bar', figsize=(10, 6))

plt.xlabel('Años') # agrega al gráfico la etiqueta del eje x
plt.ylabel('Número de inmigrantes') # agrega al gráfico la etiqueta del eje y
plt.title('Inmigrantes Islandeses a Canada desde 1980 a 2013') # agrega al gráfico el título

plt.show()

El gráfico de barras de arriba muestra el número total de inmigrantes desglosados ​​por cada año. Podemos ver claramente el impacto de la crisis financiera; el número de inmigrantes a Canadá comenzó a aumentar rápidamente después de 2008.

Anotemos esto en el gráfico usando el método `annotate` de la **capa de secuencias de comandos** o de la **interfaz pyplot**. Pasaremos los siguientes parámetros:

*   `s`: str, el texto de la anotación.
*   `xy`: Tupla que especifica el punto (x,y) de la anotación (en este caso, el punto final de la flecha).
*   `xytext`: Tupla que especifica el punto (x,y) para colocar el texto (en este caso, el punto de inicio de la flecha).
*   `xycoords`: El sistema de coordenadas en que está xy - 'data' usa el sistema de coordenadas del objetos que está siendo anotado (por defecto).
*   `arrowprops`: Toma un diccionario de propiedades para dibujar la flecha:
    *   `arrowstyle`: Especifica el estilo de la flecha, `'->'` es la flecha estándar.
    *   `connectionstyle`: Especifica el tipo de conexión. `arc3` es una línea recta.
    *   `color`: Especifica el color de la flecha.
    *   `lw`: Especifica el grosor de la línea.

Se recomienda leer la documentación de Matplotlib para obtener más detalles sobre las anotaciones:
<https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.annotate.html>.


In [ ]:
df_islandia.plot(kind='bar', figsize=(10, 6), rot=90)  # rota las marcas del eje x (puntos etiquetados en el eje x) en 90 grados

plt.xlabel('Años')
plt.ylabel('Número de inmigrantes')
plt.title('Inmigrantes Islandeses a Canada desde 1980 a 2013')

# flecha de la anotación
plt.annotate('',  # s: str. Dejar en blanco para que no haya texto
             xy=(32, 70),  # posiciona la cabeza de la flecha en el punto (alo 2012 , pobla 70)
             xytext=(28, 20),  # pociciona la base de la flecha en el punto (año 2008 , pobla 20)
             xycoords='data',  # se usará el sistema de coordenada del objeto donde se esta haciendo la anotación
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='blue', lw=2)
             )

plt.show()

También se puede poner un texto por encima de la flecha. Pasaremos los siguientes parámetros adicionales:

*   `rotation`: angulo de rotación del texto en grados (en el sentido horario)
*   `va`: alineamiento vertival del texto \[‘center’ | ‘top’ | ‘bottom’ | ‘baseline’]
*   `ha`: alineamiento horizontal del texto \[‘center’ | ‘right’ | ‘left’]


In [ ]:
df_islandia.plot(kind='bar', figsize=(10, 6), rot=90)

plt.xlabel('Años')
plt.ylabel('Número de inmigrantes')
plt.title('Inmigrantes Islandeses a Canada desde 1980 a 2013')

# flecha de la anotación
plt.annotate('',  # s: str. Dejar en blanco para que no haya texto
             xy=(32, 70),  # posiciona la cabeza de la flecha en el punto (alo 2012 , pobla 70)
             xytext=(28, 20),  # pociciona la base de la flecha en el punto (año 2008 , pobla 20)
             xycoords='data',  # se usará el sistema de coordenada del objeto donde se esta haciendo la anotación
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='blue', lw=2)
             )

# Texto de la anotación
plt.annotate('Crisis Financiera del 2008 - 2011',  # texto a mostrar
             xy=(28, 30),  # comienzo del texto en el punto (año 2008 , pobla 30)
             rotation=72.5,  # Basado en la técnica de prueba y error para calzar con la flecha
             va='bottom',  # alineación vertical
             ha='left',  # alineación horizontal
             )

plt.show()

**Gráfico de Barras Horizontal**

A veces es más práctico representar los datos horizontalmente, especialmente si se necesita más espacio para etiquetar las barras. En los gráficos de barras horizontales, el eje y se usa para etiquetar, y la longitud de las barras en el eje x corresponde a la magnitud de la variable que se mide. De este modo, hay más espacio en el eje y para etiquetar variables categóricas.

**Pregunta:** Usando la secuencia de comandos antes ocupada y el conjunto de datos `df_can`, crear un gráfico de barra *horizontal* que muestre el número *total* de inmigrantes a Canadá de los 15 países principales, para el período 1980 - 2013. Etiquete cada país con el conteo total de inmigrantes.


Paso 1: Obtener los datos pertinentes de los 15 países que más han emigrado a Canadá.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo

#La respuesta correcta es:  

# ordenar de forma descendente el dataframe basado en la columna 'Total'
df_can.sort_values(by='Total', ascending=True, inplace=True)

# rescatar los 15 países que más habitantes han emigrado a Canadá
df_top15 = df_can['Total'].tail(15)
df_top15



Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

# ordenar de forma descendente el dataframe basado en la columna 'Total'
df_can.sort_values(by='Total', ascending=True, inplace=True)

# rescatar los 15 países que más habitantes han emigrado a Canadá
df_top15 = df_can['Total'].tail(15)
df_top15

-->

Paso 2: Graficar los datos:

1.  Usar `kind='barh'` para crear el gráfico con barras horizontales.
2.  Elija un tamaño de la gráfica lo suficientemente bueno para que el gráfico, las etiquetas y el título se vean bien.
3.  Recorra los países y anote la población inmigrante utilizando la función de anotación de la interfaz de secuencias de comandos.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo





Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:  

    
# crear el gráfico
df_top15.plot(kind='barh', figsize=(12, 12), color='steelblue')
plt.xlabel('Número de Inmigrantes')
plt.title('15 Países con más Inmigrantes hacia Canadá entre 1980 - 2013')

# Anotación del número de inmigrantes por país
for indice, valor in enumerate(df_top15): 
    etiqueta = format(int(valor), ',') # formato entero con commas
    # poner el texto al final de cada barra (restando 47000 desde x, y 0.1 desde y para que calce dentro de la barra)
    plt.annotate(etiqueta, xy=(valor - 47000, indice - 0.10), color='white')

plt.show()

-->